In [3]:
import os
import glob
import mir_eval
import json
import sys
import properties as prop

In [4]:
melodia_output_path = prop.melodiaOutput

mel1_refpath = prop.annotationsRoot + "/MELODY1"

mel1_refpath = prop.annotationsRoot + "/MELODY2"

melodyJsons = prop.testDataName + "/" +  prop.melodyTrackIdsJson
with open(melodyJsons, 'r') as f:
    melody_trackids = json.load(f)
    melody_trackids = melody_trackids['tracks']
    


In [5]:
melodia_outdirs = glob.glob(os.path.join(melodia_output_path, "nu_*"))

In [6]:
melodia_outdirs

[]

In [7]:
for outdir in melodia_outdirs:
    res_path_mel1 = "melodia_scores/melody1/%s_melody1.json" % os.path.basename(outdir)
    res_path_mel2 = "melodia_scores/melody2/%s_melody2.json" % os.path.basename(outdir)
    
    if os.path.exists(res_path_mel1) or os.path.exists(res_path_mel1):
        continue
    
    mel1_results = dict.fromkeys(melody_trackids)
    mel2_results = dict.fromkeys(melody_trackids)
    
    print("Scoring for ", outdir)
    
    for track in melody_trackids:
        est_fname = "%s_MIX_vamp_mtg-melodia-nu_melodianu_melody.csv" % str(track)
        est_fpath = os.path.join(outdir, est_fname)
        est_t, est_f0 = mir_eval.io.load_time_series(est_fpath, delimiter=',')
        
        ref1_fname = "%s_MELODY1.csv" % str(track)
        ref1_fpath = os.path.join(mel1_refpath, ref1_fname)
        ref1_t, ref1_f0 = mir_eval.io.load_time_series(ref1_fpath, delimiter=',')
        
        ref2_fname = "%s_MELODY2.csv" % str(track)
        ref2_fpath = os.path.join(mel2_refpath, ref2_fname)
        ref2_t, ref2_f0 = mir_eval.io.load_time_series(ref2_fpath, delimiter=',')
        
        mel1_results[track] = mir_eval.melody.evaluate(ref1_t, ref1_f0, est_t, est_f0)
        mel2_results[track] = mir_eval.melody.evaluate(ref2_t, ref2_f0, est_t, est_f0)
        
        sys.stdout.write('.')
    
    with open(res_path_mel1, 'w') as fhandle:
        json.dump(mel1_results, fhandle)
        
    with open(res_path_mel2, 'w') as fhandle:
        json.dump(mel2_results, fhandle)
            
    print("")